# Chapter 16. Logistic Regression

In [3]:
from __future__ import division
from collections import Counter
from functools import partial  # https://www.pydanny.com/python-partials-are-fun.html
from linear_algebra import dot, vector_add
from gradient_descent import maximize_stochastic, maximize_batch
from working_with_data import rescale
from machine_learning import train_test_split
from multiple_regression import estimate_beta, predict
import math, random

In Chapter 1, we briefly looked at the problem of trying to predict which DataSciencester users paid for premium accounts.  
Here we'll revisit that problem.

## The Problem